In [1]:
import os
 
def file_name(file_dir): 
    for root, dirs, files in os.walk(file_dir):
        #print(root) #当前目录路径
        #print(dirs) #当前路径下所有子目录
        print(files) #当前路径下所有非目录子文件
    return files


## 控制c 只有需要检测的图片

In [2]:
c=file_name("testdata")

['baby_GT.bmp', 'bird_GT.bmp', 'butterfly_GT.bmp', 'head_GT.bmp', 'woman_GT.bmp']


In [3]:
for i in c:
    print(i)

baby_GT.bmp
bird_GT.bmp
butterfly_GT.bmp
head_GT.bmp
woman_GT.bmp


In [4]:
import os
filename = os.path.join("testdata/"+c[0])

In [5]:
count=len(c)
print(count)

5


# 加载模型

In [6]:
import argparse

import torch
import torch.backends.cudnn as cudnn
import numpy as np
import PIL.Image as pil_image

from temodel import SRCNN
from myutils import rgb2ycbcr, ycbcr2rgb, calc_psnr

In [7]:
cudnn.benchmark = True
device = torch.device('cuda:0' if torch.cuda.is_available() else 'cpu')

In [8]:
model = SRCNN().to(device)

In [9]:
# state_dict = model.state_dict(torch.load(mydir))

state_dict = model.state_dict()

# 定义psnr函数

In [10]:
def get_psnr(filename,myscale):
    scale=myscale
    image = pil_image.open(filename).convert('RGB')
    image_width = (image.width // scale) * scale
    image_height = (image.height // scale) * scale
    image = image.resize((image_width, image_height), resample=pil_image.BICUBIC)
    image = image.resize((image.width // scale, image.height // scale), resample=pil_image.BICUBIC)
    image = image.resize((image.width *  scale, image.height * scale), resample=pil_image.BICUBIC)
#     image.save(filename.replace('.', '_bicubic_x{}.'.format(scale)))
    image = np.array(image).astype(np.float32)
    ycbcr = rgb2ycbcr(image)

    y = ycbcr[..., 0]
    y /= 255.
    y = torch.from_numpy(y).to(device)
    y = y.unsqueeze(0).unsqueeze(0)

    with torch.no_grad():
        preds = model(y).clamp(0.0, 1.0)

    psnr = calc_psnr(y, preds)

    print('PSNR: {:.2f}'.format(psnr))

#     preds = preds.mul(255.0).cpu().numpy().squeeze(0).squeeze(0)

#     output = np.array([preds, ycbcr[..., 1], ycbcr[..., 2]]).transpose([1, 2, 0])
#     output = np.clip(ycbcr2rgb(output), 0.0, 255.0).astype(np.uint8)
#     output = pil_image.fromarray(output)
#     output.save(filename.replace('.', '_srcnn_x{}.'.format(scale)))
    
    return psnr


# 成组计算psnr

In [11]:
modeldir='./x2/'

In [12]:
myscale=2

In [13]:
mypsnr=[]

In [14]:
for i in range(400):
    modelname=modeldir+'epoch_{}.pth'.format(i)
    print(modelname)
    for n, p in torch.load(modelname, map_location=lambda storage, loc: storage).items():
        if n in state_dict.keys():
            state_dict[n].copy_(p)
        else:
            raise KeyError(n)

    model.eval()
    sum_psnr=0
    for i in range(count):
        filename = os.path.join("testdata/"+c[i])
        sum_psnr += get_psnr(filename,myscale)

    print("psnr is :{:.2f}".format(sum_psnr/count))
    mypsnr.append((sum_psnr/count).cpu())
    

./x2/epoch_0.pth


C:\Users\Administrator\AppData\Local\Temp\ipykernel_29672\3770783583.py:6: DeprecationWarning: BICUBIC is deprecated and will be removed in Pillow 10 (2023-07-01). Use Resampling.BICUBIC instead.
  image = image.resize((image_width, image_height), resample=pil_image.BICUBIC)
C:\Users\Administrator\AppData\Local\Temp\ipykernel_29672\3770783583.py:7: DeprecationWarning: BICUBIC is deprecated and will be removed in Pillow 10 (2023-07-01). Use Resampling.BICUBIC instead.
  image = image.resize((image.width // scale, image.height // scale), resample=pil_image.BICUBIC)
C:\Users\Administrator\AppData\Local\Temp\ipykernel_29672\3770783583.py:8: DeprecationWarning: BICUBIC is deprecated and will be removed in Pillow 10 (2023-07-01). Use Resampling.BICUBIC instead.
  image = image.resize((image.width *  scale, image.height * scale), resample=pil_image.BICUBIC)


PSNR: 37.22
PSNR: 34.41
PSNR: 27.07
PSNR: 38.11
PSNR: 31.16
psnr is :33.59
./x2/epoch_1.pth
PSNR: 36.88
PSNR: 34.01
PSNR: 26.08
PSNR: 37.59
PSNR: 30.66
psnr is :33.04
./x2/epoch_2.pth
PSNR: 36.47
PSNR: 33.56
PSNR: 25.45
PSNR: 37.47
PSNR: 30.31
psnr is :32.65
./x2/epoch_3.pth
PSNR: 36.13
PSNR: 33.27
PSNR: 25.11
PSNR: 37.32
PSNR: 29.97
psnr is :32.36
./x2/epoch_4.pth
PSNR: 36.23
PSNR: 33.21
PSNR: 25.02
PSNR: 37.29
PSNR: 29.98
psnr is :32.34
./x2/epoch_5.pth
PSNR: 35.98
PSNR: 32.92
PSNR: 24.77
PSNR: 37.09
PSNR: 29.60
psnr is :32.07
./x2/epoch_6.pth
PSNR: 35.58
PSNR: 32.64
PSNR: 24.62
PSNR: 36.94
PSNR: 29.28
psnr is :31.81
./x2/epoch_7.pth
PSNR: 35.88
PSNR: 32.87
PSNR: 24.81
PSNR: 37.10
PSNR: 29.41
psnr is :32.01
./x2/epoch_8.pth
PSNR: 35.69
PSNR: 32.65
PSNR: 24.62
PSNR: 36.96
PSNR: 29.20
psnr is :31.82
./x2/epoch_9.pth
PSNR: 35.88
PSNR: 32.86
PSNR: 24.77
PSNR: 37.15
PSNR: 29.20
psnr is :31.97
./x2/epoch_10.pth
PSNR: 35.28
PSNR: 32.29
PSNR: 24.25
PSNR: 36.67
PSNR: 28.70
psnr is :31.44
./x2

PSNR: 33.66
PSNR: 30.57
PSNR: 22.75
PSNR: 33.97
PSNR: 26.76
psnr is :29.54
./x2/epoch_90.pth
PSNR: 33.28
PSNR: 30.31
PSNR: 22.62
PSNR: 33.69
PSNR: 26.51
psnr is :29.28
./x2/epoch_91.pth
PSNR: 33.24
PSNR: 30.22
PSNR: 22.57
PSNR: 33.61
PSNR: 26.39
psnr is :29.20
./x2/epoch_92.pth
PSNR: 33.86
PSNR: 30.62
PSNR: 22.85
PSNR: 34.26
PSNR: 27.00
psnr is :29.72
./x2/epoch_93.pth
PSNR: 33.22
PSNR: 30.17
PSNR: 22.67
PSNR: 33.56
PSNR: 26.53
psnr is :29.23
./x2/epoch_94.pth
PSNR: 33.43
PSNR: 30.37
PSNR: 22.64
PSNR: 33.78
PSNR: 26.65
psnr is :29.37
./x2/epoch_95.pth
PSNR: 33.35
PSNR: 30.28
PSNR: 22.61
PSNR: 33.72
PSNR: 26.54
psnr is :29.30
./x2/epoch_96.pth
PSNR: 33.62
PSNR: 30.50
PSNR: 22.73
PSNR: 33.84
PSNR: 26.71
psnr is :29.48
./x2/epoch_97.pth
PSNR: 33.30
PSNR: 30.18
PSNR: 22.58
PSNR: 33.71
PSNR: 26.44
psnr is :29.24
./x2/epoch_98.pth
PSNR: 32.80
PSNR: 29.94
PSNR: 22.10
PSNR: 33.23
PSNR: 26.33
psnr is :28.88
./x2/epoch_99.pth
PSNR: 33.66
PSNR: 30.57
PSNR: 22.72
PSNR: 33.93
PSNR: 26.63
psnr is :2

PSNR: 32.90
PSNR: 29.78
PSNR: 22.32
PSNR: 33.15
PSNR: 26.31
psnr is :28.89
./x2/epoch_178.pth
PSNR: 33.20
PSNR: 29.85
PSNR: 22.42
PSNR: 33.46
PSNR: 26.46
psnr is :29.08
./x2/epoch_179.pth
PSNR: 33.34
PSNR: 30.11
PSNR: 22.43
PSNR: 33.55
PSNR: 26.65
psnr is :29.22
./x2/epoch_180.pth
PSNR: 33.33
PSNR: 30.06
PSNR: 22.56
PSNR: 33.78
PSNR: 26.54
psnr is :29.25
./x2/epoch_181.pth
PSNR: 33.43
PSNR: 30.20
PSNR: 22.80
PSNR: 33.66
PSNR: 26.76
psnr is :29.37
./x2/epoch_182.pth
PSNR: 33.18
PSNR: 29.97
PSNR: 22.40
PSNR: 33.37
PSNR: 26.37
psnr is :29.06
./x2/epoch_183.pth
PSNR: 33.17
PSNR: 29.98
PSNR: 22.53
PSNR: 33.41
PSNR: 26.44
psnr is :29.10
./x2/epoch_184.pth
PSNR: 33.11
PSNR: 29.79
PSNR: 22.22
PSNR: 33.34
PSNR: 26.36
psnr is :28.96
./x2/epoch_185.pth
PSNR: 33.10
PSNR: 29.85
PSNR: 22.42
PSNR: 33.35
PSNR: 26.42
psnr is :29.03
./x2/epoch_186.pth
PSNR: 33.02
PSNR: 29.71
PSNR: 22.31
PSNR: 33.30
PSNR: 26.36
psnr is :28.94
./x2/epoch_187.pth
PSNR: 33.08
PSNR: 29.89
PSNR: 22.31
PSNR: 33.35
PSNR: 26.39


PSNR: 33.03
PSNR: 29.62
PSNR: 22.18
PSNR: 33.33
PSNR: 26.34
psnr is :28.90
./x2/epoch_266.pth
PSNR: 32.85
PSNR: 29.56
PSNR: 22.23
PSNR: 33.13
PSNR: 26.11
psnr is :28.78
./x2/epoch_267.pth
PSNR: 32.98
PSNR: 29.70
PSNR: 22.50
PSNR: 33.29
PSNR: 26.31
psnr is :28.96
./x2/epoch_268.pth
PSNR: 33.06
PSNR: 29.69
PSNR: 22.40
PSNR: 33.33
PSNR: 26.31
psnr is :28.96
./x2/epoch_269.pth
PSNR: 32.79
PSNR: 29.49
PSNR: 22.13
PSNR: 33.10
PSNR: 26.16
psnr is :28.74
./x2/epoch_270.pth
PSNR: 33.02
PSNR: 29.79
PSNR: 22.47
PSNR: 33.23
PSNR: 26.33
psnr is :28.97
./x2/epoch_271.pth
PSNR: 32.83
PSNR: 29.58
PSNR: 22.34
PSNR: 32.98
PSNR: 26.17
psnr is :28.78
./x2/epoch_272.pth
PSNR: 33.10
PSNR: 29.73
PSNR: 22.40
PSNR: 33.34
PSNR: 26.38
psnr is :28.99
./x2/epoch_273.pth
PSNR: 33.59
PSNR: 30.15
PSNR: 22.73
PSNR: 33.69
PSNR: 26.71
psnr is :29.37
./x2/epoch_274.pth
PSNR: 32.77
PSNR: 29.48
PSNR: 22.21
PSNR: 33.15
PSNR: 26.26
psnr is :28.78
./x2/epoch_275.pth
PSNR: 32.88
PSNR: 29.63
PSNR: 22.29
PSNR: 33.23
PSNR: 26.35


PSNR: 33.26
PSNR: 29.80
PSNR: 22.58
PSNR: 33.49
PSNR: 26.41
psnr is :29.11
./x2/epoch_354.pth
PSNR: 32.84
PSNR: 29.50
PSNR: 22.16
PSNR: 33.17
PSNR: 26.05
psnr is :28.74
./x2/epoch_355.pth
PSNR: 32.89
PSNR: 29.51
PSNR: 22.42
PSNR: 33.09
PSNR: 26.17
psnr is :28.82
./x2/epoch_356.pth
PSNR: 32.89
PSNR: 29.56
PSNR: 22.30
PSNR: 33.06
PSNR: 26.15
psnr is :28.79
./x2/epoch_357.pth
PSNR: 32.80
PSNR: 29.52
PSNR: 22.29
PSNR: 32.96
PSNR: 26.02
psnr is :28.72
./x2/epoch_358.pth
PSNR: 32.86
PSNR: 29.49
PSNR: 22.26
PSNR: 33.18
PSNR: 26.18
psnr is :28.79
./x2/epoch_359.pth
PSNR: 32.60
PSNR: 29.29
PSNR: 22.03
PSNR: 32.91
PSNR: 25.85
psnr is :28.54
./x2/epoch_360.pth
PSNR: 33.05
PSNR: 29.70
PSNR: 22.43
PSNR: 33.20
PSNR: 26.30
psnr is :28.94
./x2/epoch_361.pth
PSNR: 32.78
PSNR: 29.54
PSNR: 22.19
PSNR: 33.10
PSNR: 26.07
psnr is :28.73
./x2/epoch_362.pth
PSNR: 32.77
PSNR: 29.44
PSNR: 21.98
PSNR: 33.08
PSNR: 26.06
psnr is :28.67
./x2/epoch_363.pth
PSNR: 32.94
PSNR: 29.59
PSNR: 22.42
PSNR: 33.25
PSNR: 26.24


In [15]:
import numpy

In [16]:
mypsnr=torch.from_numpy(numpy.array(mypsnr))

In [17]:
mypsnr.shape

torch.Size([400])

In [18]:
torch.save(mypsnr,"mypsnr.pth")